# Resistive Wall Wakefield

The `ResistiveWallWakefield` class implements the pseudomode (damped oscillator) fits from SLAC-PUB-10707 to efficiently calculate the short-range resistive wall wakefield for round and flat geometries.

The wakefield is represented as a single damped sinusoid:
$$W(z) = A \, e^{d z} \sin(k_r z + \phi)$$
where $A$, $d$, $k_r$, and $\phi$ are derived from polynomial fits of digitized data from Figures 4, 8, and 14 of SLAC-PUB-10707.

**References:**
- Bane & Stupakov, [SLAC-PUB-10707](https://www.slac.stanford.edu/cgi-wrap/getdoc/slac-pub-10707.pdf) (2004)
- Bane, Stupakov, Tu, [EPAC 2006 THPCH073](https://accelconf.web.cern.ch/e06/PAPERS/THPCH073.PDF)

In [ ]:
from pmd_beamphysics.wakefields import ResistiveWallWakefield
from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.units import epsilon_0

import numpy as np
import matplotlib.pyplot as plt

## Basic Usage

Create a wakefield by specifying the pipe geometry and material properties:

In [ ]:
# Create a wakefield for a 2.5 mm radius copper pipe
wake = ResistiveWallWakefield(
    radius=0.0025,
    conductivity=6.5e7,  # σ [S/m]
    relaxation_time=27e-15,  # τ [s]
    geometry="round",
)
wake

The built-in `plot()` method shows the wakefield as a function of trailing distance:

In [ ]:
wake.plot()

### Material Presets

Common materials are available via `from_material()`:

In [ ]:
# Available material presets
list(ResistiveWallWakefield.MATERIALS)

In [ ]:
wake_cu = ResistiveWallWakefield.from_material("copper-slac-pub-10707", radius=2.5e-3)
wake_cu

### Bmad Export

The wakefield can be exported in Bmad format:

In [ ]:
print(wake_cu.to_bmad())

## Validation Against SLAC-PUB-10707

Compare the pseudomode fit against digitized data from Figure 4 (AC-Cu, round pipe):

In [ ]:
# Load digitized reference data
raw_data = np.loadtxt("../data/SLAC-PUB-10707-digitized-Fig4-AC-Cu.csv", delimiter=",")
radius_ref = 2.5e-3

# Convert CGS units to SI
zref = raw_data[:, 0] * 1e-6  # µm → m
Wref = raw_data[:, 1] * 4 / radius_ref**2 / (4 * np.pi * epsilon_0)  # V/pC/m

# Create wakefield with same parameters
wake_ref = ResistiveWallWakefield.from_material(
    "copper-slac-pub-10707", radius=radius_ref
)
zlist = np.linspace(0, 300e-6, 200)
Wz = wake_ref(-zlist)  # Evaluate at trailing positions

# Plot comparison
fig, ax = plt.subplots()
ax.plot(zlist * 1e6, Wz * 1e-12, label=f"ResistiveWallWakefield ({wake_ref.geometry})")
ax.plot(zref * 1e6, Wref * 1e-12, "--", label="Fig. 4 AC-Cu (digitized)")
ax.legend()
ax.set_xlabel(r"$-z$ (µm)")
ax.set_ylabel(r"$W_z$ (V/pC/m)")
ax.set_title("Validation: Round Copper Pipe")

## Particle Wakefield Kicks

Compute the integrated wakefield kick for each particle in a bunch:

In [ ]:
# Load a particle beam
P = ParticleGroup("../data/bmad_particles2.h5")
P.drift_to_t()  # Align particles at constant time
P

The `particle_kicks` method computes the wakefield-induced energy change for each particle:

In [ ]:
# Compute wakefield kicks (eV/m) for each particle
kicks = wake_ref.particle_kicks(P)
len(kicks)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(P.z * 1e6, kicks, s=1, color="black")
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"Wakefield kick (eV/m)")
ax.set_title("Per-particle wakefield kicks")

## Convenience: `ParticleGroup.wakefield_plot`

The wakefield can be computed and plotted directly from a `ParticleGroup`. The plot automatically chooses the appropriate horizontal axis based on whether particles are aligned at constant $t$ or constant $z$:

In [ ]:
# Particles at constant t → use z as horizontal axis
P.drift_to_t()
P.wakefield_plot(wake_ref, figsize=(12, 4))

In [ ]:
# Particles at constant z → use t as horizontal axis
P.drift_to_z()
P.wakefield_plot(wake_ref, figsize=(12, 4))